In [1]:
import os
import sys
import shutil
import subprocess
sys.path.append("../")

from pathlib import Path

In [1]:
from HandsFreeDocking.tools.Chimera_Prep import run_chimera_script
from HandsFreeDocking.tools.Protein_Preparation import ProteinPreparation_Protoss, ProteinPreparation_PDBFixer, ProteinPreparation_Meeko

ModuleNotFoundError: No module named 'HandsFreeDocking.tools.Chimera_Prep'

In [3]:
from scrubber import Scrub
from\x rdkit import Chem
from rdkit.Chem import AllChem

### Scrub Trial

In [4]:
scrub = Scrub(
    ph_low=7.4,
    ph_high=7.4,
)

mol = Chem.MolFromSmiles("OC(=O)C1=CC(C=C)=CC(CC2=NN=NN2)=C1")

# each state (e.g. tautomer) an rdkit mol and may have multiple conformers
mols = []
for mol_state in scrub(mol):
    try:
        Chem.SanitizeMol(mol_state)
        mols.append(mol_state)
    except:
        continue

[15:52:45] Can't kekulize mol.  Unkekulized atoms: 15 16 17
[15:52:45] Can't kekulize mol.  Unkekulized atoms: 15 16 17
[15:52:45] Explicit valence for atom # 3 N, 4, is greater than permitted
[15:52:45] Explicit valence for atom # 3 N, 4, is greater than permitted
[15:52:45] Explicit valence for atom # 0 N, 4, is greater than permitted


### Different Protonation Methods

In [5]:
resource_dir = Path("../examples/")

initial_protein = resource_dir / "LAG3_Moloc_1.pdb"
protein_chimera = Path("protein_chimera.mol2")
protein_protoss = Path("protein_protoss.pdb")
protein_pdbfixer = Path("protein_pdbfixer.pdb")
protein_meeko = Path("protein_meeko.pdbqt")

protein_converted = Path("protein_conv.mol2")

In [6]:
run_chimera_script(initial_protein, str(protein_chimera))

'protein_chimera.mol2'

In [7]:
pp = ProteinPreparation_Meeko()
output = pp(initial_protein, protein_meeko)

@> 3277 atoms and 1 coordinate set(s) were parsed in 0.03s.


In [9]:
prep = ProteinPreparation_Protoss()
prep(initial_protein, protein_protoss)

Preparing protein with ProtoSS ...
Job 7f6af200-7f30-44fb-bdba-12476e91d54d completed with success


PosixPath('protein_protoss.pdb')

In [10]:
prep = ProteinPreparation_PDBFixer()
prep(initial_protein, protein_pdbfixer)

PosixPath('protein_pdbfixer.pdb')

In [15]:
def pdb_to_mol2(fixed_pdb, output_mol2):
    from openbabel import pybel
    import openbabel.openbabel as ob

    """
    Read the fixed PDB in Pybel, kekulize, assign Gasteiger charges,
    and write out MOL2 with atom types and partial charges.
    """
    # Pybel will auto‐kekulize and assign bond orders
    mols = list(pybel.readfile("pdb", fixed_pdb))
    if not mols:
        raise RuntimeError(f"Could not read any molecules from {fixed_pdb}")
    mol = mols[0]
    
    mol.calccharges(model="gasteiger")

    # write MOL2
    mol.write("mol2", output_mol2, overwrite=True)

In [16]:
pdb_to_mol2(str(protein_pdbfixer), str(protein_converted))

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is protein_pdbfixer.pdb)



### Meeko Based Protonation Method

In [ ]:
from prody import parsePDB, writePDB, calcCenter
from tempfile import gettempdir, NamedTemporaryFile

from pymol import cmd

In [ ]:
INIT = str(initial_protein)
MIDDLE = "protein_meeko_H.pdb"
FINAL = "protein_meeko"

In [ ]:
env = os.environ.copy()
env["MMTBX_CCP4_MONOMER_LIB"] = "/home/hitesit/Software/FORK/geostd"

In [ ]:
subprocess.run([
    "mmtbx.reduce2",
    INIT,
    "approach=add", "add_flip_movers=True" , "output.filename=" + MIDDLE, "--overwrite"
], check=True, env=env)

In [ ]:
cmd.reinitialize()
cmd.load(INIT, "Protein")
cmd.load(MIDDLE, "Protein_H")

cmd.align("Protein_H", "Protein")
cmd.delete("Protein")

cmd.load("Fake_Crystal.sdf", "Crystal")

cmd.create("Protein_Crystal", "Protein Crystal")

temp_dir = gettempdir()
temp_file_1 = f"{temp_dir}/TMP_1.pdb"

cmd.save(temp_file_1, "Protein_Crystal")

In [ ]:
atoms = parsePDB(temp_file)
receptor_atoms = atoms.select("chain A and not water and not hetero")
ligand_atoms = atoms.select("resname UNK")

temp_file_2 = f"{temp_dir}/TMP_2.pdb"
writePDB(temp_file_2, receptor_atoms)

center_x, center_y, center_z = calcCenter(ligand_atoms)
padding_x, padding_y, padding_z = (10, 10, 10)

In [ ]:
print(temp_file_2)

In [ ]:
# Like this requrie the SDF file
subprocess.run([
    "mk_prepare_receptor.py",
    "-i", temp_file_2,
    "-o", FINAL,
    "-p", "-v",
    "--box_center",
    str(center_x), str(center_y), str(center_z),
    "--box_size",
    str(padding_x), str(padding_y), str(padding_z),
    "--allow_bad_res",
], check=True)

# The -p flag says that will output a PDBQT file while the -v flag says that will output the vina box

In [ ]:
# Like this it does not requie a PDB file
subprocess.run([
    "mk_prepare_receptor.py",
    "-i", temp_file_2,
    "-o", FINAL,
    "-p",
    "--allow_bad_res",
], check=True)